In [ ]:
import json
import sys
from operator import add

from pyspark import SparkContext


def review_partitioner(key):
    return ord(key[:1])


if __name__ == '__main__':

    
    read_file = sys.argv[1]
    write_file = sys.argv[2]
    flag = sys.argv[3]
    num_partition = int(sys.argv[4])
    num_n = sys.argv[5]

    result_dict = dict()

    sc = SparkContext.getOrCreate()

    rd = sc.textFile(read_file).map(lambda row: json.loads(row))
    bd = rd.map(lambda kv: (kv['business_id'], 1))

    if flag != "default":
        bd = bd.partitionBy(num_partition, review_partitioner)

    result_dict['n_partitions'] = bd.getNumPartitions()
    result_dict['n_items'] = bd.glom().map(len).collect()
    result_dict['result'] = bd.reduceByKey(lambda a, b: a+b).filter(lambda kv: kv[1] > int(num_n)).collect()

    json.dump(result_dict, output_file)
    